Note: This was originally created on VSCode and has since been copied to Google Colab and exported to GitHub


In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# Load your dataset
df = pd.read_csv('NACA_aero_data.csv')

# Drop the Reynolds Number column
df = df.drop(columns=['Reynolds Number'])

# Display the first few rows of the dataframe
print(df.head())

In [ ]:
# Assuming all your columns are numeric and relevant for the model
X = df.drop('NACA Number', axis=1)  # Features
y = df['NACA Number']               # Target variable

# Convert target variable to categorical
y = pd.Categorical(y)

# Dictionary to turn index back into NACA
index_to_naca = dict(enumerate(y.categories))

y = y.codes

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Create a TensorFlow model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(len(set(y)), activation='softmax')  # Output layer with softmax for multi-class classification
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(),
              loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy for integer labels
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
# Save the model for future use
model.save('my_model')

In [ ]:
from tensorflow.keras.models import load_model

# Run to load the model
loaded_model = load_model('my_model')

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

In [ ]:
# Function to recommend aerofoil
def recommend_aerofoil(altitude, chord_length, mach_number, alpha, Cl, Cl_Cd):
    # Create a DataFrame for the input
    input_data = pd.DataFrame([[altitude, chord_length, mach_number, alpha, Cl, Cl_Cd]],
                              columns=['Altitude (ft)', 'Chord Length', 'Mach Number', 'Alpha', 'CL', 'CL/CD'])

    # Scale the input data using the loaded scaler
    input_scaled = scaler.transform(input_data)

    # Predict the aerofoil using the trained TensorFlow model
    predictions = model.predict(input_scaled)
    #print(predictions)
    predicted_aerofoil = tf.argmax(predictions, axis=1).numpy()[0]  # Get the index of the highest probability

    #print(predicted_aerofoil)

    naca_number = index_to_naca[predicted_aerofoil]

    # Filter the dataset for the predicted aerofoil and the given conditions
    suitable_aerofoils = df[(df['NACA Number'] == naca_number) &
                            (df['CL/CD'] >= Cl_Cd) &
                            (df['CL'] >= Cl) &
                            (df['Altitude (ft)'] == altitude) &
                            (df['Chord Length'] == chord_length) &
                            (df['Mach Number'] >= 0.98*mach_number) &
                            (df['Mach Number'] <= 1.02*mach_number) &
                            (df['Alpha'] <= alpha)
                           ]

    # Check if there are suitable aerofoils that meet all criteria
    if suitable_aerofoils.empty:
        return "No suitable aerofoil found"
    else:
        return suitable_aerofoils # Can be tweaked to return a single aerofoil if multiple match the criteria (i.e. return the one with highest Cl/Cd)


In [ ]:
# Inputs
altitude = 25000 # Feet
chord_length = 3.5 # Metres
mach_number = 0.35 # Mach
alpha = 8 # Maximum angle of attack (sum of cruising angle of attack + angle of wing on fuselage)
Cl = 1.6 # Minimum lift coefficient
Cl_Cd = 100 # Minimum lift to drag ratio


best_aerofoil = recommend_aerofoil(altitude, chord_length, mach_number, alpha, Cl, Cl_Cd)
print(f"Recommended Aerofoils:\n{best_aerofoil}")